# USGS Metadata Scrape

In [38]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
from bs4 import BeautifulSoup

### Inside the quotations in the box below, input the directory where you have saved the "SuwData" and this script file.

In [39]:
directory = "C:/Users/robert.taylor/Documents/"

## This code pulls stream, spring, and groundwater well gauge site names, site #s, locations, types of parameters, lengths of record, # of samples, etc from USGS Site Web Service within our chosen HUCs:


    Accounting Unit 031102 -- Suwannee: The Suwannee River Basin.
                              Florida, Georgia.
                                Area =     9930 sq.mi.

      Cataloging Units  03110201 -- Upper Suwannee.  Florida, Georgia.
                                      Area =     2720 sq.mi.
                        03110202 -- Alapaha. Florida, Georgia.
                                      Area =     1840 sq.mi.
                        03110203 -- withlacoochee. Florida, Georgia.
                                      Area =     1510 sq.mi.
                        03110204 -- Little. Georgia.
                                      Area =      884 sq.mi.
                        03110205 -- Lower Suwannee. Florida.
                                      Area =     1590 sq.mi.
                        03110206 -- Santa Fe. Florida.
                                      Area =     1390 sq.mi.
                                      


### Stream Gauge Sites

#### The following cell pulls the metadata for all stream gauges within the Suwannee River Basin and creates a pandas dataframe. The first 5 rows of the dataframe are displayed below.

In [45]:
# URL generated by the USGS Site Web Service. Count number is for # of daily values.
#stream_meta_full = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03110201,03110202,03110203,03110204,03110205,03110206&startDT=1980-01-01&endDT=2022-01-20&outputDataTypeCd=dv&siteType=ST&siteStatus=all", sep='\t',skiprows=(42),header=(0))
stream_meta_full = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03110206&startDT=1980-01-01&endDT=2022-01-20&outputDataTypeCd=dv&siteType=ST&siteStatus=all", sep='\t',skiprows=(42),header=(0))
stream_meta_df = stream_meta_full
stream_meta_df.drop([0],axis=0,inplace=True)
display(stream_meta_df)
stream_meta_df.to_csv(directory+'SuwData/SanteFeSites_20220121.csv')

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,02320700,"SANTA FE RIVER NEAR GRAHAM, FLA.",ST,29.84635394,-82.2195493,S,NAD83,103.55,.1,...,00003,26529,NaN,wat,NaN,1645423,0,1957-09-01,2022-01-20,19499
2,USGS,02320700,"SANTA FE RIVER NEAR GRAHAM, FLA.",ST,29.84635394,-82.2195493,S,NAD83,103.55,.1,...,00003,26530,NaN,wat,NaN,17164583,0,1957-08-07,2022-01-20,18334
3,USGS,02320700,"SANTA FE RIVER NEAR GRAHAM, FLA.",ST,29.84635394,-82.2195493,S,NAD83,103.55,.1,...,00003,300900,NaN,wat,NaN,1642503,0,2014-10-01,2022-01-20,2593
4,USGS,02321000,NEW RIVER NR LAKE BUTLER FLA,ST,29.9982954,-82.2739992,S,NAD83,83.80,.1,...,00003,26540,Tallahasee Records,wat,NaN,1645423,0,1950-01-01,2022-01-20,19082
5,USGS,02321000,NEW RIVER NR LAKE BUTLER FLA,ST,29.9982954,-82.2739992,S,NAD83,83.80,.1,...,00003,26541,Tallahasee Records,wat,NaN,17164583,0,1950-01-20,2022-01-20,17817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,USGS,02322800,SANTA FE RIVER NR HILDRETH FLA,ST,29.91162076,-82.8604033,S,NAD83,3.50,.1,...,00003,300905,NaN,wat,NaN,1642503,0,2014-10-01,2022-01-20,2661
71,USGS,02322800,SANTA FE RIVER NR HILDRETH FLA,ST,29.91162076,-82.8604033,S,NAD83,3.50,.1,...,00003,26623,NaN,wat,NaN,0,0,1978-10-01,1979-09-29,352
72,USGS,02322800,SANTA FE RIVER NR HILDRETH FLA,ST,29.91162076,-82.8604033,S,NAD83,3.50,.1,...,00001,26625,NaN,wat,NaN,701177,0,2013-07-10,2016-09-30,981
73,USGS,02322800,SANTA FE RIVER NR HILDRETH FLA,ST,29.91162076,-82.8604033,S,NAD83,3.50,.1,...,00002,26626,NaN,wat,NaN,701177,0,2013-07-10,2016-09-30,981


#### The following cell determines which sites have more than 50 daily values recorded, prints the site numbers, and counts how many sites there are that fit that description.

In [41]:
stream_meta_df['count_nu'] = stream_meta_df['count_nu'].astype(int)
stream_meta_df = stream_meta_df.loc[stream_meta_df['count_nu'] > 50]
num_sites = stream_meta_df.site_no.unique()
print(num_sites)
len(num_sites)

['02320700' '02321000' '02321500' '02321898' '02321900' '02321958'
 '02321975' '02322016' '02322049' '02322050' '02322500' '02322616'
 '02322700' '02322703' '02322800']


15

#### A list of unique parameter codes measured by the stream gauges is generated. Counts of each parameter are shown.

In [42]:
num_parms = stream_meta_df.parm_cd.unique()
print(num_parms)
stream_meta_df['parm_cd'].value_counts()

['00060' '00065' '63160' '00010' '00095' '00045' '99133' '72020']


00010    18
00095    18
00060    14
00065    11
99133     6
63160     5
72020     1
00045     1
Name: parm_cd, dtype: int64

#### Parameter codes from the USGS are pulled down from the web and saved as a dataframe. Column headings are changed. The first 5 rows of the table are shown.

In [43]:
usgs_parm_codes_df = pd.read_csv(directory + 'SuwData/USGS_All_Parameter_Codes.csv', converters={'parm_cd':lambda x: str(x)})
#usgs_parm_codes_df = usgs_parm_codes[0]
usgs_parm_codes_df.rename(columns={'Parameter Code': 'parm_cd', 
                            'Group Name': 'grp_nm', 'Parameter Name/Description':  'parm_desc', 
                            'Parameter Unit': 'parm_unit'}, inplace = True)
usgs_parm_codes_df


,Unnamed: 0,parm_cd,count_nu,grp_nm,parm_desc,CASRN,SRSName,parm_unit
0,1646,00010,118,Physical,"Temperature, water, degrees Celsius",NaN,"Temperature, water",deg C
1,1951,62611,94,Physical,"Groundwater level above NAVD 1988, feet",NaN,NaN,ft
2,2059,72019,94,Physical,"Depth to water level, feet below land surface",NaN,Depth to water level below land surface,ft
3,1950,62610,94,Physical,"Groundwater level above NGVD 1929, feet",NaN,NaN,ft
4,1692,00095,89,Physical,"Specific conductance, water, unfiltered, micro...",NaN,Specific conductance,uS/cm @25C
5,1677,00065,87,Physical,"Gage height, feet",NaN,"Height, gage",ft
6,1673,00060,67,Physical,"Discharge, cubic feet per second",NaN,"Stream flow, mean. daily",ft3/s
7,3421,99133,33,Nutrient,"Nitrate plus nitrite, water, in situ, milligra...",NaN,Inorganic nitrogen (nitrate and nitrite),mg/l as N
8,2740,00300,30,"Inorganics, Major, Non-metals","Dissolved oxygen, water, unfiltered, milligram...",7782-44-7,Oxygen,mg/l
9,1733,00400,30,Physical,"pH, water, unfiltered, field, standard units",NaN,pH,std units


In [44]:
# Use line below to find the name and description of a given parameter code.
usgs_parm_codes_df.loc[usgs_parm_codes_df['parm_cd']=='00060']

,Unnamed: 0,parm_cd,count_nu,grp_nm,parm_desc,CASRN,SRSName,parm_unit
6,1673,00060,67,Physical,"Discharge, cubic feet per second",NaN,"Stream flow, mean. daily",ft3/s


#### Stream site parameter codes are compared the the USGS master list and a table of the parameters measured at the sites is shown. 

In [34]:
avail_parms = usgs_parm_codes_df.loc[usgs_parm_codes_df['parm_cd'] == num_parms]
avail_parm_codes_stream = np.where(usgs_parm_codes_df['parm_cd'].isin(num_parms))
avail_parms_stream = usgs_parm_codes_df.iloc[avail_parm_codes_stream]
display(avail_parms_stream)
stream_meta_df['parm_cd'].value_counts()

ValueError: ('Lengths must match to compare', (20,), (8,))

#### Changing the number within the ' ' below will give a list of the stream sites that measure that parameter. Available codes to choose from are shown in table above.

In [35]:
display(stream_meta_df.loc[stream_meta_df['parm_cd'] == '00060'])

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,02320700,"SANTA FE RIVER NEAR GRAHAM, FLA.",ST,29.84635394,-82.2195493,S,NAD83,103.55,.1,...,00003,26529,NaN,wat,NaN,1645423,0,1957-09-01,2022-01-20,19499
4,USGS,02321000,NEW RIVER NR LAKE BUTLER FLA,ST,29.9982954,-82.2739992,S,NAD83,83.80,.1,...,00003,26540,Tallahasee Records,wat,NaN,1645423,0,1950-01-01,2022-01-20,19082
10,USGS,02321500,"SANTA FE RIVER AT WORTHINGTON SPRINGS, FLA.",ST,29.9219073,-82.4262246,S,NAD83,42.74,.1,...,00003,26544,NaN,wat,NaN,1645423,0,1931-10-01,2022-01-20,32985
16,USGS,02321898,SANTA FE RIVER AT O'LENO STATE PARK FLA,ST,29.92656389,-82.5596472,S,NAD83,NaN,NaN,...,00003,26556,NaN,wat,NaN,1645423,0,2011-10-01,2022-01-20,3765
18,USGS,02321900,"PARENERS BRANCH NEAR BLAND, FL.",ST,29.90746195,-82.5351166,S,NAD83,NaN,NaN,...,00003,26557,NaN,wat,NaN,1645423,0,1992-09-30,1996-10-17,1479
22,USGS,02321958,"SANTA FE RIVER AT RIVER RISE NR HIGH SPRINGS, FL",ST,29.8735723,-82.59122849,S,NAD83,31,1.6,...,00003,26565,NaN,wat,NaN,1645423,0,2015-10-01,2022-01-20,2204
30,USGS,02321975,SANTA FE RIVER AT US HWY 441 NEAR HIGH SPRINGS...,ST,29.85273894,-82.6084508,S,NAD83,35,1.6,...,00003,26566,NaN,wat,NaN,1645423,0,1992-10-01,2022-01-20,4256
35,USGS,02322016,"BLUES CREEK NEAR GAINSVILLE, FL",ST,29.72829934,-82.4314979,S,NAD83,105,.1,...,00003,26568,Tallahassee records,wat,NaN,1645423,0,1984-06-13,1994-09-29,3761
36,USGS,02322049,"BAD DOG BRANCH NEAR ALACHUA, FL",ST,29.82579759,-82.4681682,S,NAD83,NaN,NaN,...,00003,26570,NaN,wat,NaN,1645423,0,1995-10-01,2006-09-29,3956
39,USGS,02322050,"SHILOH RUN NEAR ALACHUA, FL",ST,29.81857546,-82.4723348,S,NAD83,NaN,NaN,...,00003,26572,Tallahassee records,wat,NaN,1645423,0,1983-03-04,1987-10-08,1533


### Spring Gauge Sites


#### The following cell pulls the metadata for all spring gauges within the Suwannee River Basin and creates a pandas dataframe. The first 5 rows of the dataframe are displayed below.

In [23]:
spring_meta_full = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03110201,03110202,03110203,03110204,03110205,03110206&startDT=1980-01-01&endDT=2022-01-20&outputDataTypeCd=dv&siteType=SP&siteStatus=all", sep='\t',skiprows=(42),header=(0))

spring_meta_df = spring_meta_full
spring_meta_df.drop([0],axis=0,inplace=True)
spring_meta_df.head()


,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,02315620,"HOLTON SPRINGS NEAR FT. UNION,FL",SP,30.43771795,-83.0573582,S,NAD83,NaN,NaN,...,00003,175008,NaN,wat,NaN,1645423,0,2015-06-18,2016-09-29,470
2,USGS,02315620,"HOLTON SPRINGS NEAR FT. UNION,FL",SP,30.43771795,-83.0573582,S,NAD83,NaN,NaN,...,00003,26399,NaN,wat,NaN,17164583,0,2015-06-18,2016-09-29,470
3,USGS,02319302,"MADISON BLUE SPRING NR BLUE SPRINGS, FL",SP,30.48049106,-83.2443097,F,NAD83,48,15,...,00001,222461,NaN,wat,NaN,1645597,0,2014-07-09,2022-01-19,2158
4,USGS,02319302,"MADISON BLUE SPRING NR BLUE SPRINGS, FL",SP,30.48049106,-83.2443097,F,NAD83,48,15,...,00002,222463,NaN,wat,NaN,1645597,0,2014-07-09,2022-01-19,2158
5,USGS,02319302,"MADISON BLUE SPRING NR BLUE SPRINGS, FL",SP,30.48049106,-83.2443097,F,NAD83,48,15,...,00003,172070,NaN,wat,NaN,1645597,0,2014-07-09,2022-01-19,2158


#### The following cell determines which sites have more than 10 daily values recorded, prints the site numbers, and counts how many sites there are that fit that description.

In [24]:
spring_meta_df['count_nu'] = spring_meta_df['count_nu'].astype(int)
spring_meta_df = spring_meta_df.loc[spring_meta_df['count_nu'] > 20]

num_sites = spring_meta_df.site_no.unique()
print(num_sites)
len(num_sites)

['02315620' '02319302' '02319520' '02319950' '02320250' '02322400'
 '02322685' '02322687' '02322688' '02322691' '02322694' '02322695'
 '02322698' '02323502' '02323566']


15

## Wendy Springs

In [25]:
searchfor = ['DEVIL', 'HORNSBY', 'ICHETUCKNEE', 'Ichetucknee', 'Blue Hole', 'BLUE HOLE', 'GINNIE', 'JULY']
search_results = spring_meta_df[spring_meta_df['station_nm'].str.startswith(tuple(searchfor))]
display(search_results)
print(search_results['site_no'].value_counts())
LocID = tuple(search_results['station_nm'])
print(LocID)

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
70,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00001,242605,NaN,wat,NaN,1645597,0,2018-12-14,2022-01-19,735
71,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00002,242604,NaN,wat,NaN,1645597,0,2018-12-14,2022-01-19,735
72,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00003,242602,NaN,wat,NaN,17164583,0,2018-11-09,2022-01-19,1094
73,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00001,242608,NaN,wat,NaN,1646694,0,2018-12-14,2022-01-19,742
74,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00002,242607,NaN,wat,NaN,1646694,0,2018-12-14,2022-01-19,742
75,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00003,242609,NaN,wat,NaN,1646694,0,2018-12-14,2022-01-19,742
76,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00001,242612,NaN,wat,NaN,154302,0,2018-12-14,2022-01-19,624
77,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00002,242611,NaN,wat,NaN,154302,0,2018-12-14,2022-01-19,624
78,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00003,242613,NaN,wat,NaN,154302,0,2018-12-14,2022-01-19,624
79,USGS,02322400,GINNIE SPRING NR HIGH SPRINGS FLA,SP,29.8360705,-82.7001197,S,NAD83,NaN,NaN,...,00001,242616,NaN,wat,NaN,17028275,0,2018-12-14,2022-01-19,628


02322688    18
02322400    15
02322698     2
02322694     2
02322685     2
Name: site_no, dtype: int64
('GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'GINNIE SPRING NR HIGH SPRINGS FLA', 'ICHETUCKNEE HEAD SPRING NR HILDRETH, FL', 'ICHETUCKNEE HEAD SPRING NR HILDRETH, FL', 'BLUE HOLE SPRING NR HILDRETH, FL', 'BLUE HOLE SPRING NR HILDRETH, FL', 'BLUE HOLE SPRING NR HILDRETH, FL', 'BLUE HOLE SPRING NR HILDRETH, FL', 'BLUE HOLE SPRING NR HILDRETH, FL', 'BLUE HOLE SPRING NR HILDRETH, FL', 'BLUE HOLE SPRING NR HILDRETH, FL', 'BL

#### A list of unique parameter codes measured by the spring gauges is generated. Counts of each parameter are shown.

In [26]:
num_parms = spring_meta_df.parm_cd.unique()
print(num_parms)
print(len(num_parms))
spring_meta_df['parm_cd'].value_counts()

['00060' '00065' '00010' '00095' '00300' '00400' '63160' '99133']
8


00400    24
00300    24
99133    24
00095    24
00010    23
00065    15
00060    12
63160     4
Name: parm_cd, dtype: int64

#### Spring site parameter codes are compared the the USGS master list and a table of the parameters measured at the sites is shown.

In [27]:
avail_parm_codes_spring = np.where(usgs_parm_codes_df['parm_cd'].isin(num_parms))
avail_parms_spring = usgs_parm_codes_df.iloc[avail_parm_codes_spring]
display(avail_parms_spring)

KeyError: 'parm_cd'

#### Changing the number within the ' ' below will give a list of the spring sites that measure that parameter. Available codes to choose from are shown in table above.

In [28]:
display(spring_meta_df.loc[spring_meta_df['parm_cd'] == '00060'])

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,02315620,"HOLTON SPRINGS NEAR FT. UNION,FL",SP,30.43771795,-83.0573582,S,NAD83,NaN,NaN,...,00003,175008,NaN,wat,NaN,1645423,0,2015-06-18,2016-09-29,470
6,USGS,02319302,"MADISON BLUE SPRING NR BLUE SPRINGS, FL",SP,30.48049106,-83.2443097,F,NAD83,48,15,...,00003,174138,NaN,wat,NaN,1645423,0,2002-04-12,2022-01-19,6920
40,USGS,02319950,"BLUE SPRINGS NEAR DELL,FL",SP,30.126055,-83.2259712,S,NAD83,25.00,5,...,00003,224525,NaN,wat,NaN,1645423,0,2015-06-17,2022-01-19,2409
85,USGS,02322685,"ICHETUCKNEE HEAD SPRING NR HILDRETH, FL",SP,29.98388889,-82.7619444,S,NAD83,21.76,.01,...,00003,26587,NaN,wat,NaN,1645423,0,2002-02-07,2010-06-08,3044
87,USGS,02322687,"CEDAR HEAD SPRING NR HILDRETH, FL",SP,29.98305556,-82.75888889,S,NAD83,13.83,.01,...,00003,26589,NaN,wat,NaN,1645423,0,2002-02-11,2010-05-22,3023
92,USGS,02322688,"BLUE HOLE SPRING NR HILDRETH, FL",SP,29.9799552,-82.758457,S,NAD83,20.31,.01,...,00003,229570,NaN,wat,NaN,1645423,0,2002-02-05,2022-01-19,7109
107,USGS,02322691,"MISSION SPRINGS COMPLEX NR HILDRETH, FL",SP,29.97583333,-82.7583333,S,NAD83,0.51,.01,...,00003,26612,NaN,wat,NaN,1645423,0,2002-02-12,2010-05-06,3006
109,USGS,02322694,"DEVIL'S EYE SPRING NR HILDRETH, FL",SP,29.97333333,-82.7602778,S,NAD83,12.50,.01,...,00003,26614,NaN,wat,NaN,1645423,0,2002-02-12,2010-06-09,3040
111,USGS,02322695,"MILL POND SPRING NR HILDRETH, FL",SP,29.96638889,-82.76,S,NAD83,18.76,.01,...,00003,26616,NaN,wat,NaN,1645423,0,2002-02-14,2010-04-20,2988
113,USGS,02322698,"ICHETUCKNEE R A DAMPIER'S LANDING NR HILDRETH, FL",SP,29.96027778,-82.7722222,S,NAD83,8.62,.01,...,00003,26618,NaN,wat,NaN,1645423,0,2002-02-14,2012-09-29,3751


### Groundwater Well Sites


#### The following cell pulls the metadata for all groundwater wells within the Suwannee River Basin and creates a pandas dataframe. The first 5 rows of the dataframe are displayed below.

In [29]:
gw_meta_full = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03110201,03110202,03110203,03110204,03110205,03110206&startDT=1980-01-01&endDT=2021-06-09&outputDataTypeCd=gw&siteType=GW,GW-CR,GW-EX,GW-HZ,GW-IW,GW-MW,GW-TH&siteStatus=all", sep='\t',skiprows=(42),header=(0))

gw_meta_df = gw_meta_full
gw_meta_df.drop([0],axis=0,inplace=True)
gw_meta_df.head()


,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,292448083023801,GP-6 UFA Well,GW,29.41333333,-83.04388889,S,NAD83,9,4.3,...,NaN,0,NaN,wat,NaN,1642461,0,2009-05-22,2009-05-22,1
2,USGS,292448083023801,GP-6 UFA Well,GW,29.41333333,-83.04388889,S,NAD83,9,4.3,...,NaN,0,NaN,wat,NaN,1642461,0,2009-05-22,2009-05-22,1
3,USGS,292448083023801,GP-6 UFA Well,GW,29.41333333,-83.04388889,S,NAD83,9,4.3,...,NaN,0,NaN,wat,NaN,1642008,0,2009-05-22,2009-05-22,1
4,USGS,292507082560201,A J MIMMS(121420)SR 347 SW OF CHIEFLAND,GW,29.41885004,-82.9337331,S,NAD83,33.00,.01,...,NaN,0,NaN,wat,NaN,1642461,0,1986-09-17,2009-05-19,28
5,USGS,292507082560201,A J MIMMS(121420)SR 347 SW OF CHIEFLAND,GW,29.41885004,-82.9337331,S,NAD83,33.00,.01,...,NaN,0,NaN,wat,NaN,1642461,0,1986-09-17,2009-05-19,28


#### The following cell determines which sites have more than 5 groundwater levels recorded, prints the site numbers, and counts how many sites there are that fit that description.

In [30]:
gw_meta_df['count_nu'] = gw_meta_df['count_nu'].astype(int)
gw_meta_df = gw_meta_df.loc[gw_meta_df['count_nu'] > 20]

num_sites = gw_meta_df.site_no.unique()
print(num_sites)
len(num_sites)

['292507082560201' '292713082493601' '292843082514201' '292921082583285'
 '292935083025402' '294307082020903' '294313082024601' '294330082445001'
 '294530082232001' '294538082592601' '294721082443001' '294743082543901'
 '294839082230701' '294928082355301' '295114082393801' '295214082482501'
 '295618082440901' '295707082393701' '295850082140301' '300020082103001'
 '300101082245201' '300143082565903' '300400082585001' '300612082363101'
 '300615082130501' '300823083175901' '301031082381001' '301909082490985'
 '301921082333401' '302100083171001' '302323082493501' '302450083174885'
 '302835082545301' '302847083145401' '303158082562901' '303622083050601'
 '303957083123401' '304251083230601' '304352083190501' '304446083400401'
 '304504083130501' '304603083160601' '304611083084301' '304614083140401'
 '304625083180801' '304725083241801' '304814083212101' '304849083170601'
 '304855083194701' '304942082213801' '304949083165301' '304950083113201'
 '304952083183901' '305017083195401' '3051010831815

94

#### A list of unique parameter codes measured by the groundwater sites is generated. Counts of each parameter are shown.

In [31]:
num_parms = gw_meta_df.parm_cd.unique()
print(num_parms)
print(len(num_parms))
gw_meta_df['parm_cd'].value_counts()
gw_meta_df_unique = gw_meta_df.loc[gw_meta_df['parm_cd'] == '62611']
gw_meta_df_unique.to_csv('C:/Users/robert.taylor/Documents/SuwData/USGS_Wells_df.csv')

['62610' '62611' '72019']
3


#### Groundwater site parameter codes are compared the the USGS master list and a table of the parameters measured at the sites is shown.

In [32]:
avail_parm_codes_gw = np.where(usgs_parm_codes_df['parm_cd'].isin(num_parms))
avail_parms_gw = usgs_parm_codes_df.iloc[avail_parm_codes_gw]
display(avail_parms_gw)

KeyError: 'parm_cd'

#### Changing the number within the ' ' below will give a list of the groundwater sites that measure that parameter. Available codes to choose from are shown in table above.

In [33]:
display(gw_meta_df.loc[gw_meta_df['parm_cd'] == '62610'])

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
4,USGS,292507082560201,A J MIMMS(121420)SR 347 SW OF CHIEFLAND,GW,29.41885004,-82.9337331,S,NAD83,33.00,.01,...,NaN,0,NaN,wat,NaN,1642461,0,1986-09-17,2009-05-19,28
7,USGS,292713082493601,"H.E.MILLS NR CHIEFLAND,FL",GW,29.45385208,-82.8265073,S,NAD83,31,1.6,...,NaN,0,NaN,wat,NaN,1642461,0,1984-05-15,2009-05-19,39
13,USGS,292843082514201,928251141 11S14E36 DRUMMOND LUMBER CO,GW,29.47885087,-82.86150889,S,NAD83,35.00,.1,...,NaN,0,NaN,wat,NaN,1642461,0,1961-02-08,1990-05-16,72
16,USGS,292921082583285,MANATEE SPRINGS STATE PARK,GW,29.4886611,-82.97523889,S,NAD83,16,.01,...,NaN,0,NaN,wat,NaN,1642461,0,2009-05-22,2021-12-15,63
19,USGS,292935083025402,SUNNYVALE TOWER -DUP,GW,29.49329068,-83.0481822,S,NAD83,24.23,.1,...,NaN,0,NaN,wat,NaN,1642461,0,1961-02-13,1990-05-22,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,USGS,312712082593301,18K049,GW,31.4535188,-83.49238558,F,NAD83,330.00,5,...,NaN,0,NaN,wat,NaN,1642461,0,1985-05-20,2021-12-27,175
2563,USGS,313144083335501,17L028,GW,31.529,-83.5652222,1,NAD83,322.3,10,...,NaN,0,NaN,wat,NaN,1642461,0,2001-11-30,2021-04-13,28
2569,USGS,313146083491601,15L020,GW,31.52962404,-83.8210092,F,NAD83,420,10,...,NaN,0,NaN,wat,NaN,1642461,0,1977-11-16,2021-12-13,288
2608,USGS,313233083502401,15L021,GW,31.5380111,-83.8457944,5,NAD83,382,3,...,NaN,0,NaN,wat,NaN,1642461,0,1972-01-25,2008-11-04,21


### All Sites

#### The cell below concatenates the stream, spring, and gw well metadata into one dataframe.

In [34]:
usgs_all_sites_df = pd.concat((stream_meta_df,spring_meta_df,gw_meta_df))
usgs_all_sites_df.to_csv('C:/Users/robert.taylor/Documents/SuwData/USGS_All_Sites_In_Basin.csv')

#### The following cells find the unique parameter codes in the concatenated table and then lists them in table format including the number of sites for each parameter.

In [35]:
num_parms = usgs_all_sites_df.parm_cd.unique()
print(num_parms)
print(len(num_parms))
parm_instance_all = usgs_all_sites_df['parm_cd'].value_counts()

['00060' '00045' '00065' '00010' '00095' '00300' '00400' '99133' '72192'
 '80154' '80155' '63680' '63160' '72020' '00480' '63158' '72137' '62610'
 '62611' '72019']
20


In [36]:
avail_parm_codes_all = np.where(usgs_parm_codes_df['parm_cd'].isin(num_parms))
avail_parms_all = usgs_parm_codes_df.iloc[avail_parm_codes_all]

KeyError: 'parm_cd'

In [37]:
avail_parms_count_df = pd.merge(parm_instance_all, avail_parms_all, left_index=True, right_on='parm_cd')
avail_parms_count_df = avail_parms_count_df.rename(columns={'parm_cd_x': 'count_nu'})
avail_parms_count_df = avail_parms_count_df.drop(['parm_cd_y'], axis=1)
display(avail_parms_count_df)
avail_parms_count_df.to_csv('C:/Users/robert.taylor/Documents/SuwData/USGS_All_Parameter_Codes.csv')

NameError: name 'avail_parms_all' is not defined

## All Sites within Yellow Polygon

In [38]:
gw_meta_full_1 = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03050207,03050208,03050210,03060106,03060108,03060110,03060109,03060202,03060201,03060204&startDT=1980-01-01&endDT=2021-06-27&outputDataTypeCd=gw&siteType=GW,GW-CR,GW-EX,GW-HZ,GW-IW,GW-MW,GW-TH&siteStatus=all", sep='\t',skiprows=(42),header=(0))
gw_meta_full_1.drop([0],axis=0,inplace=True)

gw_meta_full_2 = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03060203,03070107,03070106,03070102,03070105,03070104,03070202,03070203,03070201,03130006&startDT=1980-01-01&endDT=2021-06-27&outputDataTypeCd=gw&siteType=GW,GW-CR,GW-EX,GW-HZ,GW-IW,GW-MW,GW-TH&siteStatus=all", sep='\t',skiprows=(42),header=(0))
gw_meta_full_2.drop([0],axis=0,inplace=True)

gw_meta_full_3 = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03110202,03110201,03130007,03110204,03110203,03130009,03130008,03130010,03120002,03070204&startDT=1980-01-01&endDT=2021-06-27&outputDataTypeCd=gw&siteType=GW,GW-CR,GW-EX,GW-HZ,GW-IW,GW-MW,GW-TH&siteStatus=all", sep='\t',skiprows=(42),header=(0))
gw_meta_full_3.drop([0],axis=0,inplace=True)

gw_meta_full_4 = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03120003,03130013,03120001,03110103,03070205,03110102,03110205,03110206,03080103,03080201&startDT=1980-01-01&endDT=2021-06-27&outputDataTypeCd=gw&siteType=GW,GW-CR,GW-EX,GW-HZ,GW-IW,GW-MW,GW-TH&siteStatus=all", sep='\t',skiprows=(42),header=(0))
gw_meta_full_4.drop([0],axis=0,inplace=True)

gw_meta_full_5 = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03110101,03080102,03080101,03100208,03100207&startDT=1980-01-01&endDT=2021-06-27&outputDataTypeCd=gw&siteType=GW,GW-CR,GW-EX,GW-HZ,GW-IW,GW-MW,GW-TH&siteStatus=all", sep='\t',skiprows=(42),header=(0))
gw_meta_full_5.drop([0],axis=0,inplace=True)

gw_meta_exp_df = pd.concat((gw_meta_full_1,gw_meta_full_2,gw_meta_full_3,gw_meta_full_4,gw_meta_full_5))

display(gw_meta_exp_df)


,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,304450081333401,33D054 St. Marys GA no.2,GW,30.74745818,-81.5592637,1,NAD83,10,2.5,...,NaN,0,NaN,wat,NaN,1642461,0,1993-05-06,2007-05-21,29
2,USGS,304450081333401,33D054 St. Marys GA no.2,GW,30.74745818,-81.5592637,1,NAD83,10,2.5,...,NaN,0,NaN,wat,NaN,1642461,0,1993-05-06,2007-05-21,29
3,USGS,304450081333401,33D054 St. Marys GA no.2,GW,30.74745818,-81.5592637,1,NAD83,10,2.5,...,NaN,0,NaN,wat,NaN,1642008,0,1993-05-06,2007-05-21,29
4,USGS,310810081323501,"33H188 TW-26 Colonels Island, Georgia",GW,31.13634147,-81.5428798,1,NAD83,9.37,.01,...,NaN,0,NaN,wat,NaN,1642461,0,1978-12-05,2014-12-10,165
5,USGS,310810081323501,"33H188 TW-26 Colonels Island, Georgia",GW,31.13634147,-81.5428798,1,NAD83,9.37,.01,...,NaN,0,NaN,wat,NaN,1642461,0,1978-12-05,2014-12-10,165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4607,USGS,294640082064501,ROD REESE NR KEYSTONE HEIGHTS,GW,29.77802167,-82.1123236,S,NAD83,167.,.01,...,NaN,0,NaN,wat,NaN,1642461,0,1976-11-15,2007-09-17,34
4608,USGS,294640082064501,ROD REESE NR KEYSTONE HEIGHTS,GW,29.77802167,-82.1123236,S,NAD83,167.,.01,...,NaN,0,NaN,wat,NaN,1642008,0,1976-11-15,2007-09-17,34
4609,USGS,294726082101001,94721001 08S21E14 WALDO FIRE DEPT.,GW,29.79079934,-82.1692693,S,NAD83,159.00,2.5,...,NaN,0,NaN,wat,NaN,1642461,0,1977-05-03,2009-05-18,20
4610,USGS,294726082101001,94721001 08S21E14 WALDO FIRE DEPT.,GW,29.79079934,-82.1692693,S,NAD83,159.00,2.5,...,NaN,0,NaN,wat,NaN,1642461,0,1977-05-03,2009-05-18,20


In [39]:
gw_meta_exp_df['parm_cd'].value_counts()

72019    9160
62610    9158
62611    9158
00000       1
Name: parm_cd, dtype: int64

In [40]:
gw_meta_exp_df['count_nu'] = gw_meta_exp_df['count_nu'].astype(int)
gw_meta_exp_df = gw_meta_exp_df.loc[gw_meta_exp_df['count_nu'] > 10]

num_sites = gw_meta_exp_df.site_no.unique()
print(num_sites)
len(num_sites)

['304450081333401' '310810081323501' '310825081294201' ...
 '294407082262801' '294640082064501' '294726082101001']


4090

In [41]:
gw_meta_exp_df_unique = gw_meta_exp_df.loc[gw_meta_exp_df['parm_cd'] == '62611']
gw_meta_exp_df_unique.to_csv('C:/Users/robert.taylor/Documents/SuwData/USGS_Wells_All_df.csv')